# Iterativne metode


Za velike sustave, a posebno za sustave s malom ispunom (malo elemenata različitih od nule), te ukoliko je matrica sustava _strogo dijagonalno dominantna_, rješenje se može brzo naći _iterativnim metodama_
(vidi [Numerička matematika, poglavlje 3.8][RS04]):

__Definicija.__ Funkcija $F:\mathbb{R}^n\to \mathbb{R}^n$ je _kontrakcija_ ako postoji broj $q<1$ za koji vrijedi

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.
$$

__Banachov teorem o fiksnoj točki.__
Ako je $F$ kontrakcija, onda niz definiran s

$$ 
x_{k+1}=F(x_k)
$$

konvergira prema jedinstvenom vektoru $\tilde x$ za kojeg vrijedi

$$
\tilde x = F(\tilde x).
$$

$\tilde x$ se zove _fiksna točka_ funkcije $F$. Za pogrešku u $k$-tom koraku vrijede ocjene

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|
$$

i 

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,
$$

pri čemu je druga ocjena bolja. Brzina konvergencije je _linearna_,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.
$$

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."


## Jacobi-jeva i Gauss-Seidel-ova metoda

Neka je 

$$F(x)=Bx+c,$$

pri čemu je $B$ kvadratna matrica. Tada je

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

pa je $F$ kontrakcija ako je

$$
 \|B\|=q<1.
$$

Neka je zadan sustav  $Ax=b$. Matricu $A$ rastavimo kao

$$
A=D\,(L+I+U)
$$

pri čemu je $D$ dijagonalna matrica, $L$ strogo donje trokutasta matrica i $U$ strogo gornje trokutasta matrica.

### Jacobi-jeva metoda 

Neka je 

$$
B=-(L+U), \quad c=D^{-1}b.
$$


Ako je matrica $A$ _strogo dijagonalno dominantna_, 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,
$$

onda je preslikavanje $F$ kontrakcija (moguće je uzeti i druge norme) pa niz

$$
x_{k+1}=-(L+U)x_k+c
$$

konvergira prema rješenju sustava $x$.

### Gauss-Seidel-ova metoda 

Neka je 

$$
B=-(I+L)^{-1}U, \quad c=(I+L)^{-1}\, D^{-1}b.
$$

Bez dokaza navodimo sljedeću tvrdnju: ako je matrica $A$ strogo dijagonalno dominantna,
onda je preslikavanje $F$ kontrakcija pa niz

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

odnosno

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

konvergira prema rješenju sustava $x$.

In [1]:
function myjacobi(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
srand(123)
n=8
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.924965
 0.348939
 0.784423
 0.849775
 0.119929
 0.860143
 0.576808
 0.178059

In [3]:
# Početni vektor
x0=rand(n)

8-element Array{Float64,1}:
 0.951917  
 0.8781    
 0.00916625
 0.595896  
 0.954351  
 0.841945  
 0.296102  
 0.855727  

In [4]:
x,d=myjacobi(A,b,x0)
# Rješenje
x

8-element Array{Float64,1}:
  0.0898031 
  0.0143176 
  0.0745687 
  0.0821775 
 -0.00693184
  0.0811161 
  0.0568566 
 -0.00569884

In [5]:
# Norma razlike dvije zadnje iteracije
d

1.6361911825413245e-13

In [6]:
# Rezidual
r=A*x-b

8-element Array{Float64,1}:
 -3.39839e-13
 -4.47198e-13
 -3.39839e-13
 -3.29958e-13
 -3.9721e-13 
 -5.33129e-13
 -2.73004e-13
 -4.26631e-13

In [7]:
# Provjerimo i normu relativnog reziduala
norm(r)/(vecnorm(A)*norm(x))

2.629840024669059e-13

In [8]:
function mygaussseidel(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    # @show norm(U,Inf)
    y=Array{Float64}(n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [9]:
x,d=mygaussseidel(A,b,x0)
# Rješenje
x

8-element Array{Float64,1}:
  0.0898031 
  0.0143176 
  0.0745687 
  0.0821775 
 -0.00693184
  0.0811161 
  0.0568566 
 -0.00569884

In [10]:
# Norma razlike dvije zadnje iteracije
d

7.736207519235691e-14

In [11]:
# Rezidual
A*x-b

8-element Array{Float64,1}:
 -5.77316e-15
 -2.93654e-14
 -2.66454e-14
 -1.75415e-14
 -6.75848e-15
 -3.66374e-15
 -2.22045e-16
  0.0        

Izmjerimo brzinu za veće matrice:

In [12]:
n=1024
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)
# Pocetni vektor
x0=rand(n)

1024-element Array{Float64,1}:
 0.535643 
 0.766008 
 0.44635  
 0.305757 
 0.756561 
 0.159858 
 0.91704  
 0.64676  
 0.0197729
 0.297172 
 0.728198 
 0.586834 
 0.461064 
 ⋮        
 0.74365  
 0.333398 
 0.612098 
 0.237285 
 0.609827 
 0.594452 
 0.162617 
 0.357514 
 0.732225 
 0.506276 
 0.772186 
 0.881906 

In [17]:
@time mygaussseidel(A,b,x0);

  0.317902 seconds (77 allocations: 64.413 MiB, 37.88% gc time)


In [18]:
@time A\b;

  0.128247 seconds (12 allocations: 8.016 MiB, 11.35% gc time)
